# **Superstore Sales Data Cleaning**

This repository contains a project for cleaning and transforming a messy **Superstore Sales Data** dataset. The dataset includes sales records from a retail business and may contain issues such as missing values, duplicates, and inconsistent formatting, which were addressed to prepare the data for analysis and visualization.

## 1. Package Importing

To begin, we import the necessary Python packages required for data exploration and cleaning.


In [1]:
import pandas as pd
import sys
import os
import numpy as np

We also configure the environment to ensure the project directory is accessible.

In [2]:
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from superstore_sales.config import RAW_DATA_FILE, CLEANED_DATA_FILE, CLEANED_DATA_DIR
import pandas as pd

df_raw = pd.read_csv(RAW_DATA_FILE, encoding='ISO-8859-1')
df_clean = df_raw.copy()

## 2. Initial Data Exploration

To understand the dataset, we first examine its structure and content.

### 2.1 Checking Data Structure

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

### 2.2 Initial Observations

- There are no null values in the dataset.
- Some columns require a data type change:
  - Numerical values stored as `object` should be converted to `int` or `float`.
  - Date columns should be converted to `datetime`.
  - Categorical variables can be optimised using the `category` type.

In [4]:
display(df_clean.iloc[:,0:9].sample(5))
display(df_clean.iloc[:,10:].sample(5))

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country
9352,9353,CA-2017-148411,9/24/2017,9/26/2017,First Class,RO-19780,Rose O'Brian,Consumer,United States
1308,1309,CA-2016-141397,6/20/2016,6/21/2016,First Class,RC-19825,Roy Collins,Consumer,United States
1305,1306,CA-2016-101966,7/14/2016,7/16/2016,Second Class,BM-11785,Bryan Mills,Consumer,United States
6506,6507,US-2016-128909,10/9/2016,10/11/2016,Second Class,SP-20545,Sibella Parks,Corporate,United States
3798,3799,CA-2017-147760,11/4/2017,11/5/2017,First Class,KL-16555,Kelly Lampkin,Corporate,United States


,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
3103,Ohio,43229,East,OFF-BI-10001267,Office Supplies,Binders,Universal Recycled Hanging Pressboard Report B...,12.957,7,0.7,-9.5018
1036,Missouri,65109,Central,FUR-FU-10003975,Furniture,Furnishings,Eldon Advantage Chair Mats for Low to Medium P...,86.620,2,0.0,8.6620
6371,New York,10009,East,FUR-FU-10001934,Furniture,Furnishings,Magnifier Swing Arm Lamp,83.920,4,0.0,21.8192
4791,Colorado,80906,West,OFF-PA-10001166,Office Supplies,Paper,Xerox 1932,85.056,3,0.2,28.7064
8677,Texas,76063,Central,FUR-TA-10004607,Furniture,Tables,Hon 2111 Invitation Series Straight Table,517.405,5,0.3,-81.3065


In [5]:
df_clean.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [6]:
df_clean.describe(include='object')

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name
count,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994,9994
unique,5009,1237,1334,4,793,793,3,1,531,49,4,1862,3,17,1850
top,CA-2017-100111,9/5/2016,12/16/2015,Standard Class,WB-21850,William Brown,Consumer,United States,New York City,California,West,OFF-PA-10001970,Office Supplies,Binders,Staple envelope
freq,14,38,35,5968,37,37,5191,9994,915,2001,3203,19,6026,1523,48


#### Observations from Data Exploration

1. **Data Sampling:**
   - Two separate random samples were displayed: one from columns 0 to 9 and another from column 10 onwards.
   - This method allows for a quick overview of different sections of the dataset.

2. **Summary Statistics (`df_clean.describe()`):**
   - The dataset contains 9,994 entries.
   - **Sales, Profit, and Discount:**
     - Sales have a wide range, from a minimum of 0.44 to a maximum of 22,638.48.
     - Profit values vary significantly, from -6,599.98 to 8,399.98, indicating potential losses and gains.
     - Discounts range from 0 to 0.8, showing varying discount strategies.
   - **Quantity Distribution:**
     - The quantity per transaction varies from 1 to 14, with a median of 3.
   - **Postal Code Analysis:**
     - The mean postal code is around 55,190, with significant variation (std = 32,063), indicating geographic diversity in the data.

3. **Categorical Data Summary:**
   - The dataset includes categorical fields such as `Order ID`, `Customer ID`, `Product ID`, `Region`, `State`, `City`, `Category`, `Sub-Category`, and `Ship Mode`.
   - Unique counts reveal that there are 5,009 distinct orders, suggesting repeat customers or multi-product orders.
   - The presence of unique customer IDs implies customer-level tracking.

4. **Potential Areas for Further Investigation:**
   - The large standard deviation in profit suggests significant variability in product performance.
   - The presence of negative profits needs further exploration—certain products or regions may be underperforming.
   - Sales and discount correlation analysis could provide insights into pricing strategies.


In [7]:
df_clean.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

### 2.3 Data Validation Check  

The values for the following transformations were verified to ensure correctness and consistency:  

- **Row ID and Postal Code**: Converted to string format and confirmed to have no incorrect or missing values. Postal codes were checked to ensure they follow a uniform 5-digit format.  
- **Order Date and Ship Date**: Successfully converted to datetime format, with no invalid or misformatted entries.  
- **Categorical Columns**: Verified that 'Ship Mode', 'Segment', 'Country', 'Region', 'Category', and 'Sub-Category' contain only valid and expected values. No unexpected categories or misclassified data were found.  

All transformations were validated, and the data is clean and ready for further analysis.

In [8]:
# Change numbers to strings
df_clean[['Row ID', 'Postal Code']] = df_clean[['Row ID', 'Postal Code']].astype('str')

# Fill the postal codes with leading zeros to ensure a uniform 5-digit format
df_clean['Postal Code'] = df_clean['Postal Code'].str.zfill(5)

# Convert date columns to datetime format
df_clean[['Order Date', 'Ship Date']] = df_clean[['Order Date', 'Ship Date']].apply(pd.to_datetime)

# Convert selected columns to categorical data types
df_clean['Ship Mode'] = pd.Categorical(df_clean['Ship Mode'], categories=df_clean['Ship Mode'].unique(), ordered=False)
df_clean['Segment'] = pd.Categorical(df_clean['Segment'], categories=df_clean['Segment'].unique(), ordered=False)
df_clean['Country'] = pd.Categorical(df_clean['Country'], categories=['United States', 'International'], ordered=False)
df_clean['State'] = pd.Categorical(df_clean['State'], categories=df_clean['State'].unique(), ordered=False)
df_clean['Order ID'] = pd.Categorical(df_clean['Order ID'])  
df_clean['Customer ID'] = pd.Categorical(df_clean['Customer ID'])  
df_clean['Postal Code'] = pd.Categorical(df_clean['Postal Code'])  


# Ensure 'Region' is stored as a categorical variable
df_clean['Region'] = pd.Categorical(df_clean['Region'], categories=df_clean['Region'].unique(), ordered=False)

# Convert 'Category' and 'Sub-Category' to categorical types
df_clean['Category'] = pd.Categorical(df_clean['Category'], categories=df_clean['Category'].unique(), ordered=False)
df_clean['Sub-Category'] = pd.Categorical(df_clean['Sub-Category'], categories=df_clean['Sub-Category'].unique(), ordered=False)

# Check for one-to-one relationships between IDs or other key columns

# Verify if each 'Sub-Category' belongs to only one 'Category'
subcat_cat_count = df_clean.groupby('Sub-Category', observed=True)['Category'].nunique()  # Count unique categories for each sub-category
display(subcat_cat_count)  # If any count is greater than 1, the sub-category appears in multiple categories

Sub-Category
Bookcases      1
Chairs         1
Labels         1
Tables         1
Storage        1
Furnishings    1
Art            1
Phones         1
Binders        1
Appliances     1
Paper          1
Accessories    1
Envelopes      1
Fasteners      1
Supplies       1
Machines       1
Copiers        1
Name: Category, dtype: int64

#### Verification of 'Sub-Category' Consistency  

A check was performed to ensure that each **'Sub-Category'** belongs to only one **'Category'**.  
The analysis confirmed that no 'Sub-Category' appears in more than one 'Category'.  
This validation ensures the data maintains a strict hierarchical relationship between categories and sub-categories.

## 3. Cleaning data

### 3.1 Functions for Identifying and Updating Duplicate IDs
#### `dup_flag(df, id_col, value_col)`
This function identifies whether an ID appears with multiple unique values in the specified column. It returns:
- A boolean Series indicating which rows have duplicate IDs.
- A DataFrame containing unique combinations of `id_col` and `value_col`.

This is useful for detecting inconsistencies in datasets where each ID should ideally map to a single value.

#### `update_id(df, id_col, value_col)`
This function appends a numerical suffix to duplicate IDs, ensuring uniqueness while maintaining traceability. It:
- Calls `dup_flag()` to identify duplicates.
- Assigns a numerical suffix to duplicate occurrences.
- Updates the ID column by appending the suffix only for duplicates.
- Merges the updated IDs back into the original DataFrame.

This function helps standardize datasets by ensuring IDs remain unique while preserving their original structure.


In [9]:
def dup_flag(df, id_col, value_col):
    """
    Identifies duplicate IDs based on their associated values.

    Args:
        df (pd.DataFrame): The input DataFrame.
        id_col (str): Column containing IDs.
        value_col (str): Column to check for uniqueness within each ID.

    Returns:
        dup_flags (pd.Series): Boolean Series indicating which rows have duplicate IDs.
        unique_combinations (pd.DataFrame): DataFrame with unique (id_col, value_col) combinations.
    """
    unique_combinations = df[[id_col, value_col]].drop_duplicates().copy()
    dup_prod = df.groupby(id_col)[value_col].nunique()
    dup_ids = dup_prod[dup_prod > 1].index.to_list()
    dup_flags = df[id_col].isin(dup_ids)
    return dup_flags, unique_combinations

def update_id(df, id_col, value_col):
    """
    Updates duplicate IDs by appending a numerical suffix.

    Args:
        df (pd.DataFrame): The input DataFrame.
        id_col (str): Column containing IDs.
        value_col (str): Column to check for uniqueness within each ID.

    Returns:
        pd.DataFrame: DataFrame with updated IDs and duplication flags.
    """
    flags = dup_flag(df, id_col, value_col)
    df[id_col + ' dup'] = flags[0]  # Boolean flag for duplicate IDs
    suffixes = flags[1]  # DataFrame with unique ID-value combinations
    suffixes[id_col + ' suffix'] = suffixes.groupby(id_col).cumcount() + 1  # Assign incremental suffix
    new_col = id_col + ' updated'
    suffixes[new_col] = suffixes[id_col].astype(str) + "_" + suffixes[id_col + ' suffix'].astype(str).str.zfill(2)
    
    # Drop redundant columns if they exist
    df = df.drop(columns=[col for col in [id_col + ' suffix', new_col] if col in df.columns], axis=1)
    
    # Merge updated suffixes with original DataFrame
    df_new = df.merge(suffixes, on=[id_col, value_col], how='left')
    
    # Keep original ID where no duplicates exist
    df_new[new_col] = np.where(df_new[id_col + " dup"], df_new[new_col], df_new[id_col])

    return df_new


### 3.2 Handling Duplicate IDs  

The focus was placed on the **Product ID**, where duplicate IDs were detected across different products. An updated ID was generated only for those products with duplicates, ensuring that necessary updates were applied efficiently.

As for the **Customer ID**, after running `update_id(df_clean, 'Customer ID', 'Customer Name')`, the results showed that the **Customer ID** requires no modification.

In [10]:
df_clean=update_id(df_clean,'Product ID', 'Product Name')
df_clean['Product ID updated'] = pd.Categorical(df_clean['Product ID updated'])  
df_clean.drop(columns=['Product ID suffix'], inplace=True)

### 3.3 Time Series Analysis and Corrections  

- **Date Range Verification:**  
  - The dataset contains **Order Dates** from **2014-01-03** to **2017-12-30**.  
  - **Ship Dates** range from **2014-01-07** to **2018-01-05**, ensuring all orders have valid shipping records.  

In [11]:
# Check the Date Range
min_order, max_order = df_clean['Order Date'].min(), df_clean['Order Date'].max()
min_ship, max_ship = df_clean['Ship Date'].min(), df_clean['Ship Date'].max()

print(f"Order Date Range:   {min_order.strftime('%Y-%m-%d')}  to  {max_order.strftime('%Y-%m-%d')}")
print(f" Ship Date Range:   { min_ship.strftime('%Y-%m-%d')}  to  {max_ship.strftime('%Y-%m-%d')}")

Order Date Range:   2014-01-03  to  2017-12-30
 Ship Date Range:   2014-01-07  to  2018-01-05


- **Invalid Date Check:**  
  - No instances were found where an order was shipped before it was placed. This confirms data integrity in shipping timelines.  



In [12]:
# Check for Orders Shipped Before They Were Ordered
df_invalid_dates = df_clean[df_clean['Ship Date'] < df_clean['Order Date']]

if df_invalid_dates.empty:
    print("All shipping dates are valid. No orders were shipped before the order date.")
else:
    print("Orders with invalid shipping dates found:")
    print(df_invalid_dates)


All shipping dates are valid. No orders were shipped before the order date.


- **Shipping Duration Analysis:**  
  - The average shipping time is **~4 days**, with a minimum of **0 days** (same-day shipping) and a maximum of **7 days**.  
  - The distribution appears reasonable, with no extreme outliers affecting the dataset.  

In [13]:
#Check for Outliers (Extremely Long Shipping Times)
df_clean['Shipping Duration'] = df_clean['Ship Date'] - df_clean['Order Date']
print("\n📊 Shipping Duration Stats (in days):")
print(df_clean['Shipping Duration'].dt.days.describe())


📊 Shipping Duration Stats (in days):
count    9994.000000
mean        3.958175
std         1.747567
min         0.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: Shipping Duration, dtype: float64


## 4. Saving Cleaned Superstore Sales Data in Multiple Formats

This script ensures that the cleaned **Superstore Sales** dataset is saved in multiple formats—CSV, JSON, Parquet, and Pickle—allowing flexibility for future data processing and analysis.  

### 4.1 Functionality  
1. **Creates the Cleaned Data Directory**  
   - Uses `CLEANED_DATA_DIR` from the `superstore_sales.config` module.  
   - Ensures the directory exists before saving files.  

2. **Defines File Paths**  
   - Specifies locations for storing the cleaned dataset and its metadata.  
   - File types include:
     - CSV (`SuperStoreOrders_clean.csv`)
     - JSON (`SuperStoreOrders_clean_dtypes.json`)
     - Parquet (`SuperStoreOrders_clean.parquet`)
     - Pickle (`SuperStoreOrders_clean.pkl`)  

3. **Saves the Cleaned Data in Different Formats**  
   - **CSV:** A widely supported format, but does not preserve data types.  
   - **JSON:** Stores column data types to restore them when reloading the CSV.  
   - **Parquet:** Optimized for fast loading and maintains data types.  
   - **Pickle:** Fully preserves the dataset but is Python-specific.  

#### Output Messages  
Each save operation prints a message confirming that the file has been successfully written.  

In [14]:
from superstore_sales.config import CLEANED_DATA_DIR


os.makedirs(CLEANED_DATA_DIR, exist_ok=True)

# Define file paths
csv_file = os.path.join(CLEANED_DATA_DIR, 'SuperStoreOrders_clean.csv')
dtype_file = os.path.join(CLEANED_DATA_DIR, 'SuperStoreOrders_clean_dtypes.json')
parquet_file = os.path.join(CLEANED_DATA_DIR, 'SuperStoreOrders_clean.parquet')
pickle_file = os.path.join(CLEANED_DATA_DIR, 'SuperStoreOrders_clean.pkl')

# Save as CSV
df_clean.to_csv(csv_file, index=False)
print("Data saved in CSV format.")

# Save column data types as JSON for CSV restoration
df_clean.dtypes.apply(lambda x: str(x)).to_json(dtype_file)
print("Data saved in json format.")

# Save as Parquet
df_clean.to_parquet(parquet_file)
print("Data saved in Parquet format.")

# Save as Pickle
df_clean.to_pickle(pickle_file)
print("Data saved in Pickle format.")


Data saved in CSV format.
Data saved in json format.
Data saved in Parquet format.
Data saved in Pickle format.


# Comprehensive Conclusion: Superstore Sales Data Cleaning and Preparation

The **Superstore Sales** dataset has been successfully cleaned and structured, ensuring accuracy, consistency, and reliability for further analysis.  

## 1 Key Achievements  

### 1.1 Data Integrity and Structure  
- The dataset contains **9,994 records** with **21 attributes**, covering order details, customer information, product categories, sales figures, and regional data.  
- **No missing values** were found, confirming data completeness.  
- **Data types** were corrected for numerical, categorical, and date-related attributes to enhance processing efficiency.  

### 1.2. Data Cleaning and Transformation  
- **Inconsistent formatting** in numeric and date columns was rectified.  
- **Categorical variables** were optimized to improve memory efficiency.  
- **Duplicate Product IDs** were identified and resolved by assigning unique suffixes where necessary.  
- **Time series validation** ensured all orders had valid shipping dates, with no cases of shipments occurring before order placement.  

### 1.3. Descriptive Statistics and Insights  
- **Sales** ranged from **\$0.44** to **\$22,638.48**, highlighting significant variance in transaction values.  
- **Profit** variability was notable, spanning from **-\$6,599.98** (losses) to \$8,399.98 (gains).  
- **Shipping durations** were analyzed, revealing an average of **4 days**, with no extreme outliers.  

### 1.4. Data Storage and Export  
- The cleaned dataset was **saved in multiple formats** (`CSV`, `JSON`, `Parquet`, `Pickle`) to ensure flexibility in future processing and analysis.  

## 2 Next Steps  
- Conduct **exploratory data analysis (EDA)** to uncover trends in sales, profit margins, and customer segments.  
- Perform **correlation analysis** to examine relationships between sales, discounts, and profit.  
- Develop **data visualizations and reports** to extract actionable insights for business decision-making.  

The dataset is now well-prepared for advanced analysis and further investigation. 
